In [ ]:
#Bigger ANN model 4 Hidden layers amd 1 output layer
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import LeakyReLU
from glob import glob

# Constants for tire degradation
SOFT_DEG = 0.01
MEDIUM_DEG = 0.007
HARD_DEG = 0.005
NUM_LAPS = 58  # Number of laps for prediction
BATCH_SIZE = 5000  # Number of samples per training batch

# Path to stored CSV data (Update this path to your dataset location)
DATA_FOLDER = "E:/fastf1_csv_data/Data/Abu Dhabi Grand Prix/Race"

def load_offline_data():
    """Loads lap data from stored CSV files."""
    all_files = glob(os.path.join(DATA_FOLDER, "*.csv"))  # Get all CSV files
    df_list = []  # Store each CSV's data

    for file in all_files:
        df = pd.read_csv(file)
        df_list.append(df)

    return pd.concat(df_list, ignore_index=True)  # Merge all CSVs into one DataFrame

def prepare_training_data(df):
    """Processes offline lap data into training features and targets."""
    df = df.dropna(subset=['LapTime', 'Compound'])  # Remove missing lap times
    df['LapTime'] = pd.to_timedelta(df['LapTime']).dt.total_seconds()  # Convert to seconds

    soft_data = df[df['Compound'] == 'SOFT']
    medium_data = df[df['Compound'] == 'MEDIUM']
    hard_data = df[df['Compound'] == 'HARD']

    def process_compound_data(compound_data):
        lap_times = compound_data['LapTime'].values
        laps = np.arange(len(lap_times)).reshape(-1, 1)  # Use lap numbers as a feature
        return laps, lap_times  # X (features), y (targets)

    # Process each compound
    soft_X, soft_y = process_compound_data(soft_data)
    medium_X, medium_y = process_compound_data(medium_data)
    hard_X, hard_y = process_compound_data(hard_data)

    return (soft_X, soft_y), (medium_X, medium_y), (hard_X, hard_y)

def build_model():
    """Builds and compiles the ANN model."""
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(512, activation=LeakyReLU(alpha=0.1), input_shape=(1,)), # Lap number + Degradation
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(128,activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(32, activation=LeakyReLU(alpha=0.1)),
        tf.keras.layers.Dense(1)  # Output: Lap Time
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss='mse',
                  metrics=['mae'])
    return model

# Load offline data
df = load_offline_data()

# Prepare data in batches
(soft_X, soft_y), (medium_X, medium_y), (hard_X, hard_y) = prepare_training_data(df)

# Train in batches
soft_model = build_model()
medium_model = build_model()
hard_model = build_model()

def train_model_in_batches(model, X, y, batch_size):
    """Trains the model using batch processing to optimize memory usage."""
    num_samples = len(X)
    for start in range(0, num_samples, batch_size):
        end = min(start + batch_size, num_samples)
        model.fit(X[start:end], y[start:end], epochs=300, verbose=1)  # Train on batch

# Train models in batches
print("\nTraining Soft Tire Model...")
train_model_in_batches(soft_model, soft_X, soft_y, BATCH_SIZE)

print("\nTraining Medium Tire Model...")
train_model_in_batches(medium_model, medium_X, medium_y, BATCH_SIZE)

print("\nTraining Hard Tire Model...")
train_model_in_batches(hard_model, hard_X, hard_y, BATCH_SIZE)

# Predict lap times
def predict_lap_times(tire_type, initial_lap):
    """Predicts lap times for a given tire type over multiple laps."""
    if tire_type == "soft":
        model, degradation = soft_model, SOFT_DEG
    elif tire_type == "medium":
        model, degradation = medium_model, MEDIUM_DEG
    elif tire_type == "hard":
        model, degradation = hard_model, HARD_DEG
    else:
        raise ValueError("Invalid tire type. Choose from 'soft', 'medium', or 'hard'.")

    lap_times = []
    lap_number = np.array([[initial_lap]])

    for lap in range(NUM_LAPS):
        lap_time = model.predict(lap_number, verbose=0)[0][0]
        lap_times.append(lap_time)
        lap_number[0, 0] += 1  # Increment lap number

    return lap_times

def seconds_to_min_sec(predicted_lap_times):
    return[f"{int(time // 60)}:{time % 60:.3f}" for time in predicted_lap_times]

# Example prediction
initial_lap = 1
tire_type = "hard"
predicted_lap_times = predict_lap_times(tire_type, initial_lap)
formatted_lap_times = seconds_to_min_sec(predicted_lap_times)
print("\nPredicted Lap Times (MM:SS.mmm):")
for i, lap_time in enumerate(formatted_lap_times, start=1):
    print(f"Lap {i}: {lap_time}")



In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import LeakyReLU
from glob import glob

# Constants
BATCH_SIZE = 10000
NUM_LAPS = 58
DATA_FOLDER = "E:/fastf1_csv_data/Data/Belgian Grand Prix/Race"

# Function to Load Offline CSV Data
def load_offline_data():
    """Loads lap data from stored CSV files."""
    all_files = glob(os.path.join(DATA_FOLDER, '*.csv'))
    df_list = []

    for file in all_files:
        df = pd.read_csv(file)
        df_list.append(df)

    return pd.concat(df_list, ignore_index=True) if df_list else pd.DataFrame()  # Merge all CSVs into one DataFrame

# Function to Calculate Lap Time Degradation
def calculate_degradation(lap_times):
    """Calculates degradation as the difference between consecutive lap times."""
    degradation = np.diff(lap_times, prepend=lap_times[0])  # First lap has no degradation
    return degradation

# Function to Prepare Data
def prepare_training_data(df):
    """Processes offline lap data into training features and targets."""
    df = df.dropna(subset=['LapTime', 'Compound'])  # Remove missing lap times
    df['LapTime'] = pd.to_timedelta(df['LapTime']).dt.total_seconds()  # Convert LapTime to seconds

    soft_data = df[df['Compound'] == 'SOFT']
    medium_data = df[df['Compound'] == 'MEDIUM']
    hard_data = df[df['Compound'] == 'HARD']

    def process_compound_data(compound_data):
        lap_times = compound_data['LapTime'].values
        degradation = calculate_degradation(lap_times)
        laps = np.arange(1, len(lap_times) + 1).reshape(-1, 1)  # Lap number as feature
        return np.hstack((laps, degradation.reshape(-1, 1))), lap_times

    soft_X, soft_y = process_compound_data(soft_data)
    medium_X, medium_y = process_compound_data(medium_data)
    hard_X, hard_y = process_compound_data(hard_data)

    return (soft_X, soft_y), (medium_X, medium_y), (hard_X, hard_y)

# Function to Build Model
def build_model():
    """Builds and compiles the ANN model."""
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(512, activation=LeakyReLU(alpha=0.1), input_shape=(2,)), # Lap number + Degradation
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(32, activation=LeakyReLU(alpha=0.1)),
        tf.keras.layers.Dense(1)  # Output: Lap Time
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss='mse',
                  metrics=['mae'])
    return model

# Function to Train Model in Batches
def train_model_in_batches(model, X, y, batch_size):
    """Trains the model using batch processing to optimize memory usage."""
    num_samples = len(X)
    for start in range(0, num_samples, batch_size):
        end = min(start + batch_size, num_samples)
        model.fit(X[start:end], y[start:end], epochs=200, verbose=1, batch_size=32)

# Function to Predict Lap Times
def predict_lap_times(model, initial_lap, initial_lap_time):
    """Predicts lap times dynamically using calculated degradation."""
    lap_times = []
    lap_number = initial_lap
    last_lap_time = initial_lap_time

    for _ in range(NUM_LAPS):
        # Create input feature (Lap Number, Last Lap Degradation)
        degradation = last_lap_time - lap_times[-1] if lap_times else 0
        input_features = np.array([[lap_number, degradation]])

        # Predict next lap time
        lap_time = model.predict(input_features, verbose=0)[0][0]
        lap_times.append(lap_time)

        # Update values for next lap
        lap_number += 1
        last_lap_time = lap_time  # Update last lap time

    return lap_times

# Convert Seconds to MM:SS.mmm Format
def seconds_to_min_sec(predicted_lap_times):
    """Converts lap times from seconds to MM:SS.mmm format."""
    return [f"{int(time // 60)}:{time % 60:.3f}" for time in predicted_lap_times]

# Function to Select Model Based on Tire Type
def tire_type_model(tire_type):
    if tire_type == "soft":
        return soft_model
    elif tire_type == "medium":
        return medium_model
    elif tire_type == "hard":
        return hard_model
    else:
        raise ValueError("Invalid tire type")

# Load Offline Data
df = load_offline_data()
if df.empty:
    print("⚠ No offline data found! Check your CSV files.")
else:
    # Prepare Data
    (soft_X, soft_y), (medium_X, medium_y), (hard_X, hard_y) = prepare_training_data(df)

    # Build and Train Models
    soft_model = build_model()
    medium_model = build_model()
    hard_model = build_model()

    print('\n Training Soft Model...')
    train_model_in_batches(soft_model, soft_X, soft_y, BATCH_SIZE)

    print('\n Training Medium Model...')
    train_model_in_batches(medium_model, medium_X, medium_y, BATCH_SIZE)

    print('\n Training Hard Model...')
    train_model_in_batches(hard_model, hard_X, hard_y, BATCH_SIZE)

    # Predict Lap Times
    initial_lap = 1
    tire_type = "hard"
    model = tire_type_model(tire_type)
    initial_lap_time = hard_y[0]  # Start with the first lap time

    predicted_lap_times = predict_lap_times(model, initial_lap, initial_lap_time)

    # Convert to Minutes:Seconds Format
    formatted_lap_times = seconds_to_min_sec(predicted_lap_times)

    print("\nPredicted Lap Times (MM:SS.mmm):")
    for i, lap_time in enumerate(formatted_lap_times, start=1):
        print(f"Lap {i}: {lap_time}")


In [ ]:
session = fastf1.get_session(2019, 'Bahrain','Q')
session.load(telemetry=True, laps=True, weather=True)
lap_data = session.laps
print(lap_data.head())
lap_data.to_csv('lap_data_table.csv', index = True)
print("Lap data has been saved")

In [ ]:
session.get_driver('HAM')
session.load(telemetry=True, laps=True, weather=True)
session.laps
session = fastf1.get_session(2019, 'Bahrain','R')
session.load(telemetry=True, laps=True, weather=True)
lap_data = session.laps
print(lap_data.head())

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import LeakyReLU, LSTM
from glob import glob

# Constants
BATCH_SIZE = 5000
NUM_LAPS_TO_PREDICT = 58  # Number of laps to predict
DATA_FOLDER = "E:/fastf1_csv_data/Data/Belgian Grand Prix/Race"

# Fuel & Track Temp Inputs
INITIAL_FUEL_LOAD = 105  # Full F1 race fuel load (kg)
FUEL_BURN_RATE = 1.5  # Estimated fuel loss per lap (kg)
INITIAL_TRACK_TEMP = 35  # Track temperature in Celsius
TEMP_DECREASE_LAPS = 10  # Decrease track temp by 1°C every 10 laps

# **Tire Degradation Parameters**
DEGRADATION_FACTORS = {
    "SOFT": (0.07, 2.0),
    "MEDIUM": (0.05, 1.5),
    "HARD": (0.03, 1.2)
}

# **Load Offline CSV Data**
def load_offline_data():
    all_files = glob(os.path.join(DATA_FOLDER, '*.csv'))
    df_list = [pd.read_csv(file) for file in all_files]
    return pd.concat(df_list, ignore_index=True) if df_list else pd.DataFrame()

# **Calculate Tire Degradation**
def calculate_degradation(lap_numbers, compound):
    """Degradation is based on different tire compounds."""
    k1, k2 = DEGRADATION_FACTORS[compound]
    return k1 * np.log(k2 * lap_numbers + 1)

# **Prepare Training Data (For All Drivers & Compounds)**
def prepare_training_data(df):
    df = df.dropna(subset=['LapTime', 'Compound', 'Driver'])
    df['LapTime'] = pd.to_timedelta(df['LapTime']).dt.total_seconds()

    compound_data = {"SOFT": [], "MEDIUM": [], "HARD": []}

    for driver in df["Driver"].unique():
        driver_df = df[df["Driver"] == driver]

        for compound in DEGRADATION_FACTORS.keys():
            compound_df = driver_df[driver_df["Compound"] == compound]
            if compound_df.empty:
                continue

            lap_numbers = np.arange(1, len(compound_df) + 1).reshape(-1, 1)
            degradation = calculate_degradation(lap_numbers, compound)
            lap_times = compound_df['LapTime'].values

            fuel_load = np.array([max(INITIAL_FUEL_LOAD - (i * FUEL_BURN_RATE), 0) for i in range(len(lap_times))]).reshape(-1, 1)
            track_temp = np.array([INITIAL_TRACK_TEMP - (i // TEMP_DECREASE_LAPS) for i in range(len(lap_times))]).reshape(-1, 1)

            compound_data[compound].append((np.hstack((lap_numbers, degradation.reshape(-1, 1), fuel_load, track_temp)), lap_times))

    # Merge all drivers' data for each compound
    for compound in compound_data:
        if compound_data[compound]:
            X_all, y_all = zip(*compound_data[compound])
            compound_data[compound] = (np.vstack(X_all), np.concatenate(y_all))
        else:
            compound_data[compound] = (None, None)  # No data for this compound

    return compound_data

# **Build LSTM Model**
def build_model():
    model = tf.keras.Sequential([
        LSTM(128, return_sequences=True, input_shape=(None, 4)),  # Features: Lap Number, Degradation, Fuel, Temp
        tf.keras.layers.BatchNormalization(),
        LSTM(64, return_sequences=False),
        tf.keras.layers.Dense(32, activation=LeakyReLU(alpha=0.1)),
        tf.keras.layers.Dense(1)  # Output: Lap Time
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss='mse',
                  metrics=['mae'])
    return model

# **Train Model**
def train_model(model, X_train, y_train):
    X_train = X_train.reshape((X_train.shape[0], 1, 4))  # Reshape for LSTM
    model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=1)
    return model

# **Predict Multiple Laps Sequentially for a Tire Compound**
def predict_multiple_laps(model, compound, start_lap, num_laps):
    lap_times = []
    lap_number = start_lap
    last_lap_time = None

    for _ in range(num_laps):
        fuel_load = max(INITIAL_FUEL_LOAD - (lap_number * FUEL_BURN_RATE), 0)
        track_temp = INITIAL_TRACK_TEMP - (lap_number // TEMP_DECREASE_LAPS)

        # Calculate degradation for this compound
        degradation = calculate_degradation(np.array([lap_number]), compound)[0]

        # Create input features
        input_features = np.array([[lap_number, degradation, fuel_load, track_temp]]).reshape((1, 1, 4))

        # Predict lap time
        predicted_time = model.predict(input_features, verbose=0)[0][0]

        # If there's a last lap reference, apply penalty if deviation > 5 sec
        if last_lap_time is not None and abs(predicted_time - last_lap_time) > 5:
            predicted_time = last_lap_time + np.sign(predicted_time - last_lap_time) * 5

        lap_times.append(predicted_time)
        last_lap_time = predicted_time  # Store last lap time
        lap_number += 1

    return lap_times

# **Convert Lap Time to MM:SS.mmm**
def seconds_to_min_sec(lap_time):
    return f"{int(lap_time // 60)}:{lap_time % 60:.3f}"

# **Load Data & Train the Model**
df = load_offline_data()
if not df.empty:
    # **Prepare Data for Each Compound**
    compound_data = prepare_training_data(df)

    models = {}
    for compound, (X_data, y_data) in compound_data.items():
        if X_data is not None:
            print(f"\nTraining {compound} Model...")
            model = build_model()
            models[compound] = train_model(model, X_data, y_data)

    # **Predict Laps for Each Compound**
    print("\nPredicted Lap Times:")
    for compound, model in models.items():
        print(f"\n{compound} Tire Predictions:")
        predicted_lap_times = predict_multiple_laps(model, compound, start_lap=1, num_laps=NUM_LAPS_TO_PREDICT)
        formatted_times = [seconds_to_min_sec(t) for t in predicted_lap_times]

        for i, lap_time in enumerate(formatted_times, start=1):
            print(f"Lap {i}: {lap_time}")


In [11]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LeakyReLU, LSTM
from glob import glob
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, precision_score, recall_score, f1_score

# Constants
BATCH_SIZE = 5000
MODEL_SAVE_PATH = r"E:\fastf1_csv_data\models\Australian Grand Prix.h5"
NUM_LAPS_TO_PREDICT = 58  # Number of laps to predict
DATA_FOLDER = r"E:\fastf1_csv_data\Data\Australian Grand Prix\Race"

# Fuel & Track Temp Inputs
INITIAL_FUEL_LOAD = 110  # Full F1 race fuel load (kg)
FUEL_BURN_RATE = 1.5  # Estimated fuel loss per lap (kg)
INITIAL_TRACK_TEMP = 35  # Track temperature in Celsius
TEMP_DECREASE_LAPS = 10  # Decrease track temp by 1°C every 10 laps

# **Tire Degradation Parameters**
DEGRADATION_FACTORS = {
    "SOFT": (0.07, 2.0),
    "MEDIUM": (0.05, 1.5),
    "HARD": (0.03, 1.2)
}

# **Load Offline CSV Data**
def load_offline_data():
    all_files = glob(os.path.join(DATA_FOLDER, '*.csv'))
    df_list = [pd.read_csv(file) for file in all_files]
    return pd.concat(df_list, ignore_index=True) if df_list else pd.DataFrame()

# **Calculate Tire Degradation**
def calculate_degradation(lap_numbers, compound):
    """Degradation is based on different tire compounds."""
    k1, k2 = DEGRADATION_FACTORS[compound]
    return k1 * np.log(k2 * lap_numbers + 1)

# **Prepare Training Data (For All Drivers & Compounds)**
def prepare_training_data(df):
    df = df.dropna(subset=['LapTime', 'Compound', 'Driver'])
    df['LapTime'] = pd.to_timedelta(df['LapTime']).dt.total_seconds()

    compound_data = {"SOFT": [], "MEDIUM": [], "HARD": []}

    for driver in df["Driver"].unique():
        driver_df = df[df["Driver"] == driver]

        for compound in DEGRADATION_FACTORS.keys():
            compound_df = driver_df[driver_df["Compound"] == compound]
            if compound_df.empty:
                continue

            lap_numbers = np.arange(1, len(compound_df) + 1).reshape(-1, 1)
            degradation = calculate_degradation(lap_numbers, compound)
            lap_times = compound_df['LapTime'].values

            fuel_load = np.array([max(INITIAL_FUEL_LOAD - (i * FUEL_BURN_RATE), 0) for i in range(len(lap_times))]).reshape(-1, 1)
            track_temp = np.array([INITIAL_TRACK_TEMP - (i // TEMP_DECREASE_LAPS) for i in range(len(lap_times))]).reshape(-1, 1)

            compound_data[compound].append((np.hstack((lap_numbers, degradation.reshape(-1, 1), fuel_load, track_temp)), lap_times))

    # Merge all drivers' data for each compound
    for compound in compound_data:
        if compound_data[compound]:
            X_all, y_all = zip(*compound_data[compound])
            compound_data[compound] = (np.vstack(X_all), np.concatenate(y_all))
        else:
            compound_data[compound] = (None, None)  # No data for this compound

    return compound_data

# **Build LSTM Model**
def build_model():
    model = tf.keras.Sequential([
        LSTM(128, return_sequences=True, input_shape=(None, 4)),  # Features: Lap Number, Degradation, Fuel, Temp
        tf.keras.layers.BatchNormalization(),
        LSTM(64, return_sequences=False),
        tf.keras.layers.Dense(32, activation=LeakyReLU(alpha=0.1)),
        tf.keras.layers.Dense(1)  # Output: Lap Time
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss='mse',
                  metrics=['mae'])
    return model

# **Train Model**
def train_model(model, X_train, y_train):
    X_train = X_train.reshape((X_train.shape[0], 1, 4))  # Reshape for LSTM
    model.fit(X_train, y_train, epochs=500, batch_size=100, verbose=1)
    model.save(MODEL_SAVE_PATH)
    return model

# **Predict Multiple Laps Sequentially for a Tire Compound**
def predict_multiple_laps(model, compound, start_lap, num_laps):
    lap_times = []
    lap_number = start_lap
    last_lap_time = None

    for _ in range(num_laps):
        fuel_load = max(INITIAL_FUEL_LOAD - (lap_number * FUEL_BURN_RATE), 0)
        track_temp = INITIAL_TRACK_TEMP - (lap_number // TEMP_DECREASE_LAPS)

        # Calculate degradation for this compound
        degradation = calculate_degradation(np.array([lap_number]), compound)[0]

        # Create input features
        input_features = np.array([[lap_number, degradation, fuel_load, track_temp]]).reshape((1, 1, 4))

        # Predict lap time
        predicted_time = model.predict(input_features, verbose=0)[0][0]

        # If there's a last lap reference, apply penalty if deviation > 5 sec
        if last_lap_time is not None and abs(predicted_time - last_lap_time) > 5:
            predicted_time = last_lap_time + np.sign(predicted_time - last_lap_time) * 5

        lap_times.append(predicted_time)
        last_lap_time = predicted_time  # Store last lap time
        lap_number += 1

    return lap_times

# **Convert Lap Time to MM:SS.mmm**
def seconds_to_min_sec(lap_time):
    return f"{int(lap_time // 60)}:{lap_time % 60:.3f}"

# **Load Data & Train the Model**
df = load_offline_data()
if not df.empty:
    # **Prepare Data for Each Compound**
    compound_data = prepare_training_data(df)

    models = {}
    for compound, (X_data, y_data) in compound_data.items():
        if X_data is not None:
            print(f"\nTraining {compound} Model...")
            model = build_model()
            models[compound] = train_model(model, X_data, y_data)

    # **Predict Laps for Each Compound**
    print("\nPredicted Lap Times:")
    for compound, model in models.items():
        print(f"\n{compound} Tire Predictions:")
        predicted_lap_times = predict_multiple_laps(model, compound, start_lap=1, num_laps=NUM_LAPS_TO_PREDICT)
        formatted_times = [seconds_to_min_sec(t) for t in predicted_lap_times]

        for i, lap_time in enumerate(formatted_times, start=1):
            print(f"Lap {i}: {lap_time}")
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# **Function to Calculate Model Metrics**
def evaluate_model(y_true, y_pred):
    """Calculates accuracy metrics for the model."""
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)

    print(f"\n Model Performance Metrics:")
    print(f" MAE: {mae:.3f} seconds")
    print(f" MSE: {mse:.3f}")
    print(f" RMSE: {rmse:.3f} seconds")
    print(f" R² Score: {r2:.3f}")

    return mae, mse, rmse, r2

# **Test Model Performance for Each Compound**
for compound, model in models.items():
    print(f"\n Evaluating {compound} Tire Model...")
    
    # **Get True Values (y_actual)**
    X_test, y_actual = compound_data[compound]
    X_test = X_test.reshape((X_test.shape[0], 1, 4))  # Reshape for LSTM

    # **Get Model Predictions (y_predicted)**
    y_predicted = model.predict(X_test, verbose=0).flatten()

    # **Evaluate the Model**
    evaluate_model(y_actual, y_predicted)
from sklearn.metrics import precision_score, recall_score, f1_score

# **Function to Convert Lap Times into Categories**
def classify_lap_times(y_actual, y_pred):
    """Classifies lap times as 'Fast' (1) or 'Slow' (0) based on median time."""
    threshold = np.median(y_actual)  # Use median lap time as cutoff
    y_actual_class = (y_actual <= threshold).astype(int)
    y_pred_class = (y_pred <= threshold).astype(int)
    return y_actual_class, y_pred_class

# **Function to Calculate Classification Metrics**
def evaluate_classification(y_actual, y_pred):
    y_actual_class, y_pred_class = classify_lap_times(y_actual, y_pred)
    
    precision = precision_score(y_actual_class, y_pred_class)
    recall = recall_score(y_actual_class, y_pred_class)
    f1 = f1_score(y_actual_class, y_pred_class)

    print(f"\n Classification Performance:")
    print(f" Precision: {precision:.3f}")
    print(f" Recall: {recall:.3f}")
    print(f" F1 Score: {f1:.3f}")

    return precision, recall, f1

# **Run Classification Evaluation**
for compound, model in models.items():
    print(f"\n Evaluating {compound} Tire Model (Classification)...")

    # **Get True & Predicted Values**
    X_test, y_actual = compound_data[compound]
    X_test = X_test.reshape((X_test.shape[0], 1, 4))
    y_predicted = model.predict(X_test, verbose=0).flatten()

    # **Evaluate Classification Performance**
    evaluate_classification(y_actual, y_predicted)



NameError: name 'models' is not defined

In [ ]:
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.losses import MeanSquaredError

# **Define the saved model path**
MODEL_SAVE_PATH = "E:/fastf1_csv_data/models/2024_Austrian Grand Prix_Race.h5"

# **Load model with custom objects**
model = load_model(MODEL_SAVE_PATH, custom_objects={"LeakyReLU": LeakyReLU, "mse":MeanSquaredError()})
print("✅ Model loaded successfully!")

# **Constants (Same as during training)**
INITIAL_FUEL_LOAD = 110  # kg
FUEL_BURN_RATE = 1.5  # kg per lap
INITIAL_TRACK_TEMP = 35  # Celsius
TEMP_DECREASE_LAPS = 10  # Temp decreases every 10 laps
LAPS = 58

# **Function to Calculate Tire Degradation**
def calculate_degradation(lap_numbers, compound):
    """Degradation is based on different tire compounds."""
    DEGRADATION_FACTORS = {
        "SOFT": (0.07, 2.0),
        "MEDIUM": (0.05, 1.5),
        "HARD": (0.03, 1.2)
    }
    k1, k2 = DEGRADATION_FACTORS[compound]
    return k1 * np.log(k2 * lap_numbers + 1)

# **Convert Lap Time to MM:SS.mmm**
def seconds_to_min_sec(lap_time):
    """Convert seconds to MM:SS.mmm format."""
    return f"{int(lap_time // 60)}:{lap_time % 60:.3f}"

# **Function to Predict a Single Lap Time**
def predict_lap_time(model, lap_number, compound):
    """Predicts lap time for a given lap and tire compound."""
    degradation = calculate_degradation(np.array([lap_number]), compound)[0]
    fuel_load = max(INITIAL_FUEL_LOAD - (lap_number * FUEL_BURN_RATE), 0)
    track_temp = INITIAL_TRACK_TEMP - (lap_number // TEMP_DECREASE_LAPS)

    input_features = np.array([[lap_number, degradation, fuel_load, track_temp]]).reshape((1, 1, 4))
    predicted_time = model.predict(input_features, verbose=0)[0][0]

    return predicted_time

# **Predict a Single Lap Time (Example: Lap 58 using HARD tires)**
predicted_time = predict_lap_time(model, lap_number=58, compound="HARD")
formatted_time = seconds_to_min_sec(predicted_time)
print(f"🏎️ Predicted Lap Time for Lap 58 (HARD tires): {formatted_time}")

# **Function to Predict Multiple Laps**
def predict_multiple_laps(model, compound, start_lap=1, num_laps=LAPS):
    """Predicts lap times for multiple laps sequentially."""
    lap_times = []
    for lap_number in range(start_lap, start_lap + num_laps):
        predicted_time = predict_lap_time(model, lap_number, compound)
        lap_times.append(predicted_time)

    return lap_times

# **Predict the First 20 Laps for HARD Tires**
predicted_laps = predict_multiple_laps(model, compound="MEDIUM", start_lap=1, num_laps=LAPS)

# **Print Formatted Results**
print("\n🏎️ Predicted Lap Times for HARD Tires:")
for i, time in enumerate(predicted_laps, start=1):
    print(f"Lap {i}: {seconds_to_min_sec(time)}")


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model

model_path = "E:/fastf1_csv_data/models/2024_Bahrain Grand Prix_Race.h5"
model = load_model(model_path, compile=False)

print("Model Input Shape:", model.input_shape)


In [5]:
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.losses import MeanSquaredError


MODEL_SAVE_PATH = "E:/fastf1_csv_data/models/2024_Bahrain Grand Prix_Race.h5"

# **Load model with custom objects**
model = load_model(MODEL_SAVE_PATH, custom_objects={"LeakyReLU": LeakyReLU, "mse":MeanSquaredError()})
print("✅ Model loaded successfully!")

# **Constants (Same as during training)**
INITIAL_FUEL_LOAD = 110  # kg
FUEL_BURN_RATE = 1.5  # kg per lap
INITIAL_TRACK_TEMP = 35  # Celsius
TEMP_DECREASE_LAPS = 10  # Temp decreases every 10 laps
LAPS = 58

def predict_lap_time(model, lap_number, compound):
    """Predicts lap time for a given lap and tire compound."""
    degradation = calculate_degradation(np.array([lap_number]), compound)[0]
    fuel_load = max(INITIAL_FUEL_LOAD - (lap_number * FUEL_BURN_RATE), 0)
    track_temp = INITIAL_TRACK_TEMP - (lap_number // TEMP_DECREASE_LAPS)

    input_features = np.array([[lap_number, degradation, fuel_load, track_temp]]).reshape((1, 1, 4))
    predicted_time = model.predict(input_features, verbose=0)[0][0]

    return predicted_time
def seconds_to_min_sec(lap_time):
    """Convert seconds to MM:SS.mmm format."""
    return f"{int(lap_time // 60)}:{lap_time % 60:.3f}"
    
def predict_multiple_laps(model, compound, start_lap=1, num_laps=LAPS):
    """Predicts lap times for multiple laps sequentially."""
    lap_times = []
    for lap_number in range(start_lap, start_lap + num_laps):
        predicted_time = predict_lap_time(model, lap_number, compound)
        lap_times.append(predicted_time)

    return lap_times

predicted_laps = predict_multiple_laps(model, compound="HARD", start_lap=1, num_laps=LAPS)
print("\n🏎️ Predicted Lap Times for HARD Tires:")
for i, time in enumerate(predicted_laps, start=1):
    print(f"Lap {i}: {seconds_to_min_sec(time)}")

✅ Model loaded successfully!

🏎️ Predicted Lap Times for HARD Tires:
Lap 1: 1:33.987
Lap 2: 1:35.704
Lap 3: 1:35.296
Lap 4: 1:34.697
Lap 5: 1:34.530
Lap 6: 1:34.712
Lap 7: 1:35.154
Lap 8: 1:35.583
Lap 9: 1:35.894
Lap 10: 1:35.377
Lap 11: 1:35.733
Lap 12: 1:36.156
Lap 13: 1:36.660
Lap 14: 1:37.519
Lap 15: 1:38.550
Lap 16: 1:39.187
Lap 17: 1:39.842
Lap 18: 1:41.897
Lap 19: 1:43.934
Lap 20: 1:42.914
Lap 21: 1:41.404
Lap 22: 1:40.003
Lap 23: 1:39.189
Lap 24: 1:38.813
Lap 25: 1:38.817
Lap 26: 1:39.086
Lap 27: 1:39.462
Lap 28: 1:39.820
Lap 29: 1:40.110
Lap 30: 1:39.708
Lap 31: 1:39.970
Lap 32: 1:40.189
Lap 33: 1:40.397
Lap 34: 1:40.586
Lap 35: 1:40.735
Lap 36: 1:40.830
Lap 37: 1:40.876
Lap 38: 1:40.891
Lap 39: 1:40.893
Lap 40: 1:40.403
Lap 41: 1:40.439
Lap 42: 1:40.493
Lap 43: 1:40.568
Lap 44: 1:40.660
Lap 45: 1:40.764
Lap 46: 1:40.872
Lap 47: 1:40.970
Lap 48: 1:41.041
Lap 49: 1:41.057
Lap 50: 1:40.573
Lap 51: 1:40.344
Lap 52: 1:39.907
Lap 53: 1:39.288
Lap 54: 1:38.721
Lap 55: 1:38.592
Lap 5

In [3]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, LeakyReLU, BatchNormalization
from glob import glob
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, precision_score, recall_score, f1_score

# **Set Random Seeds for Stability**
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

# **Constants**
EVENT_NAME = "Azerbaijan Grand Prix"
SESSION_NAME = "Race"
YEAR = "2024"
MODEL_SAVE_PATH = f"E:/fastf1_csv_data/models/{YEAR}_{EVENT_NAME}_{SESSION_NAME}.h5"

BATCH_SIZE = 5000
NUM_LAPS_TO_PREDICT = 57
DATA_FOLDER = r"E:\fastf1_csv_data\Data\Azerbaijan Grand Prix"

# **Fuel & Track Temp Inputs**
INITIAL_FUEL_LOAD = 110  # Max fuel (kg)
FUEL_BURN_RATE = 1.5  # kg per lap
INITIAL_TRACK_TEMP = 35  # Track temperature in Celsius
TEMP_DECREASE_LAPS = 10  # Decrease track temp by 1°C every 10 laps

# **Tire Wear Limits**
TIRE_LIFESPAN = {"SOFT": 25, "MEDIUM": 35, "HARD": 45}

# **Tire Degradation Parameters**
DEGRADATION_FACTORS = {
    "SOFT": (0.07, 2.0),
    "MEDIUM": (0.05, 1.5),
    "HARD": (0.03, 1.2)
}

# **Load CSV Data**
def load_offline_data():
    all_files = glob(os.path.join(DATA_FOLDER, '*.csv'))
    df_list = [pd.read_csv(file) for file in all_files]
    return pd.concat(df_list, ignore_index=True) if df_list else pd.DataFrame()

# **Calculate Tire Degradation**
def calculate_degradation(lap_numbers, compound):
    k1, k2 = DEGRADATION_FACTORS[compound]
    return k1 * np.log(k2 * lap_numbers + 1)

# **Prepare Training Data**
def prepare_training_data(df):
    df = df.dropna(subset=['LapTime', 'Compound', 'Driver'])
    df['LapTime'] = pd.to_timedelta(df['LapTime']).dt.total_seconds()

    compound_data = {"SOFT": [], "MEDIUM": [], "HARD": []}

    for driver in df["Driver"].unique():
        driver_df = df[df["Driver"] == driver]

        for compound in DEGRADATION_FACTORS.keys():
            compound_df = driver_df[driver_df["Compound"] == compound]
            if compound_df.empty:
                continue

            lap_numbers = np.arange(1, len(compound_df) + 1).reshape(-1, 1)
            degradation = calculate_degradation(lap_numbers, compound)
            lap_times = compound_df['LapTime'].values

            fuel_load = np.array([max(INITIAL_FUEL_LOAD - (i * FUEL_BURN_RATE), 0) for i in range(len(lap_times))]).reshape(-1, 1)
            track_temp = np.array([INITIAL_TRACK_TEMP - (i // TEMP_DECREASE_LAPS) for i in range(len(lap_times))]).reshape(-1, 1)

            compound_data[compound].append((np.hstack((lap_numbers, degradation.reshape(-1, 1), fuel_load, track_temp)), lap_times))

    for compound in compound_data:
        if compound_data[compound]:
            X_all, y_all = zip(*compound_data[compound])
            compound_data[compound] = (np.vstack(X_all), np.concatenate(y_all))
        else:
            compound_data[compound] = (None, None)

    return compound_data

# **Build LSTM Model**
def build_model():
    model = Sequential([
        LSTM(128, return_sequences=True, input_shape=(None, 4)),
        BatchNormalization(),
        LSTM(64, return_sequences=False),
        Dense(32, activation=LeakyReLU(negative_slope=0.1)),
        Dense(1)
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss='mse',
                  metrics=['mae'])
    return model

# **Train Model & Save It**
def train_model(model, X_train, y_train):
    X_train = X_train.reshape((X_train.shape[0], 1, 4))
    model.fit(X_train, y_train, epochs=300, batch_size=32, verbose=1)
    model.save(MODEL_SAVE_PATH)
    return model

# **Predict Multiple Laps**
def predict_multiple_laps(model, compound, start_lap, num_laps):
    lap_times = []
    lap_number = start_lap
    last_lap_time = None

    for _ in range(num_laps):
        fuel_load = max(INITIAL_FUEL_LOAD - (lap_number * FUEL_BURN_RATE), 0)
        track_temp = INITIAL_TRACK_TEMP - (lap_number // TEMP_DECREASE_LAPS)
        degradation = calculate_degradation(np.array([lap_number]), compound)[0]

        if lap_number > TIRE_LIFESPAN[compound]:
            degradation *= 2.5  # Extreme degradation beyond max lifespan
            print(f"⚠️ {compound} tire exceeded lifespan ({TIRE_LIFESPAN[compound]} laps). Expect slower lap times!")

        input_features = np.array([[lap_number, degradation, fuel_load, track_temp]]).reshape((1, 1, 4))
        predicted_time = model.predict(input_features, verbose=0)[0][0]

        if last_lap_time is not None and abs(predicted_time - last_lap_time) > 5:
            predicted_time = last_lap_time + np.sign(predicted_time - last_lap_time) * 5

        lap_times.append(predicted_time)
        last_lap_time = predicted_time
        lap_number += 1

    return lap_times

# **Convert Lap Time to MM:SS.mmm**
def seconds_to_min_sec(lap_time):
    return f"{int(lap_time // 60)}:{lap_time % 60:.3f}"

# **Load Data & Train**
df = load_offline_data()
if not df.empty:
    compound_data = prepare_training_data(df)
    models = {}

    for compound, (X_data, y_data) in compound_data.items():
        if X_data is not None:
            print(f"\n🚗 Training {compound} Model...")
            model = build_model()
            models[compound] = train_model(model, X_data, y_data)

    # **Predict Lap Times**
    print("\n🏎️ Predicted Lap Times:")
    for compound, model in models.items():
        print(f"\n🔹 {compound} Tire Predictions:")
        predicted_lap_times = predict_multiple_laps(model, compound, start_lap=1, num_laps=NUM_LAPS_TO_PREDICT)
        formatted_times = [seconds_to_min_sec(t) for t in predicted_lap_times]

        for i, lap_time in enumerate(formatted_times, start=1):
            print(f"Lap {i}: {lap_time}")

        # **Evaluate Model**
        X_test, y_actual = compound_data[compound]
        y_predicted = model.predict(X_test.reshape((X_test.shape[0], 1, 4)), verbose=0).flatten()
        evaluate_model(y_actual, y_predicted)
        evaluate_classification(y_actual, y_predicted)


In [ ]:
pip install --upgrade ipykernel